<a href="https://colab.research.google.com/github/bintangnabiil/Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-TensorFlow/blob/main/Rangkuman_Chapter_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chapter 15: Processing Sequences Using RNNs and CNNs
Chapter ini membahas tentang pemrosesan data sekuensial menggunakan Recurrent Neural Networks (RNNs) dan Convolutional Neural Networks (CNNs). Data sekuensial adalah data yang memiliki urutan temporal seperti time series, teks, audio, dan video.

##1. Recurrent Neural Networks (RNNs)
###Konsep Dasar RNN
RNN adalah jenis neural network yang dirancang khusus untuk memproses data sekuensial. Berbeda dengan feedforward neural networks yang memproses input secara independen, RNN memiliki "memori" yang memungkinkannya mengingat informasi dari langkah waktu sebelumnya.

###Karakteristik utama RNN:

Memiliki koneksi rekuren yang membentuk loop
Dapat memproses sekuens dengan panjang yang bervariasi
Berbagi parameter antar langkah waktu
Output pada waktu t bergantung pada input saat ini dan state tersembunyi dari waktu sebelumnya

###Arsitektur RNN
RNN memiliki dua input utama:

- Input sequence (x_t): data pada waktu t
- Hidden state (h_t-1): informasi dari langkah waktu sebelumnya

Formula matematika RNN:
$$
h_t = tanh(W_xh * x_t + W_hh * h_t-1 + b_h)
$$

$$
y_t = W_hy * h_t + b_y
$$


##Implementasi RNN Sederhana

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Membuat data time series sederhana untuk demonstrasi
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20))
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)
    return series[..., np.newaxis].astype(np.float32)

# Generate training data
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

print(f"Training data shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")

Training data shape: (7000, 50, 1)
Training labels shape: (7000, 1)


##Simple RNN Model

In [ ]:
# Model RNN sederhana untuk prediksi time series
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# Training model
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid),
                    verbose=1)

##2. Masalah Vanishing Gradient pada RNN
###Penjelasan Masalah
RNN standar mengalami masalah vanishing gradient ketika memproses sekuens yang panjang. Ini terjadi karena:

- Gradient yang dipropagasi mundur menjadi semakin kecil
- Informasi dari langkah waktu yang jauh menjadi hilang
- Model sulit mempelajari dependensi jangka panjang
<br><br>

###Solusi: LSTM dan GRU
###Long Short-Term Memory (LSTM)
LSTM mengatasi masalah vanishing gradient dengan menggunakan:

- Cell state: jalur informasi jangka panjang
- Forget gate: menentukan informasi mana yang akan dilupakan
- Input gate: menentukan informasi baru yang akan disimpan
- Output gate: menentukan bagian mana dari cell state yang akan dioutput

In [ ]:
# LSTM Model
lstm_model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.LSTM(20),
    keras.layers.Dense(1)
])

lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.summary()

# Training LSTM
lstm_history = lstm_model.fit(X_train, y_train, epochs=20,
                              validation_data=(X_valid, y_valid),
                              verbose=1)

###Gated Recurrent Unit (GRU)
GRU adalah versi sederhana dari LSTM dengan:

- Update gate: menggabungkan forget dan input gate
- Reset gate: menentukan seberapa banyak informasi masa lalu yang diabaikan
- Lebih sederhana dan cepat dibandingkan LSTM

In [ ]:
# GRU Model
gru_model = keras.models.Sequential([
    keras.layers.GRU(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20),
    keras.layers.Dense(1)
])

gru_model.compile(optimizer='adam', loss='mse')

# Training GRU
gru_history = gru_model.fit(X_train, y_train, epochs=20,
                            validation_data=(X_valid, y_valid),
                            verbose=1)

##3. Convolutional Neural Networks untuk Sequences
###CNN 1D untuk Data Sekuensial
CNN tidak hanya berguna untuk data gambar, tetapi juga dapat digunakan untuk data sekuensial dengan menggunakan konvolusi 1D.

###Keuntungan CNN 1D:

- Lebih cepat dibandingkan RNN
- Dapat mendeteksi pola lokal dalam sekuens
- Parallelizable (dapat diproses secara paralel)
- Tidak mengalami masalah vanishing gradient

In [ ]:
# CNN 1D Model untuk time series
cnn_model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid",
                        input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20),
    keras.layers.Dense(1)
])

cnn_model.compile(optimizer='adam', loss='mse')
cnn_model.summary()

# Training CNN+GRU hybrid model
cnn_history = cnn_model.fit(X_train, y_train, epochs=20,
                            validation_data=(X_valid, y_valid),
                            verbose=1)

###WaveNet Architecture
WaveNet menggunakan dilated causal convolutions untuk memproses audio sequences:


In [ ]:
# Implementasi sederhana WaveNet-style model
def wavenet_residual_block(inputs, n_filters, dilation_rate):
    z = keras.layers.Conv1D(n_filters, 2, padding="causal",
                            dilation_rate=dilation_rate)(inputs)
    z = keras.layers.Activation("tanh")(z)
    z = keras.layers.Conv1D(n_filters, 1)(z)
    return keras.layers.Add()([z, inputs]), z

# Build WaveNet model
def build_wavenet_model(n_filters=32, n_blocks=3):
    inputs = keras.layers.Input(shape=[None, 1])
    z = keras.layers.Conv1D(n_filters, 2, padding="causal")(inputs)

    skip_connections = []
    for dilation_rate in [1, 2, 4, 8] * n_blocks:
        z, skip = wavenet_residual_block(z, n_filters, dilation_rate)
        skip_connections.append(skip)

    z = keras.layers.Activation("relu")(keras.layers.Add()(skip_connections))
    z = keras.layers.Conv1D(n_filters, 1, activation="relu")(z)
    outputs = keras.layers.Conv1D(1, 1)(z)

    return keras.models.Model(inputs, outputs)

wavenet_model = build_wavenet_model()
wavenet_model.compile(optimizer='adam', loss='mse')

##4. Bidirectional RNNs
###Konsep Bidirectional RNN
Bidirectional RNN memproses sekuens dalam dua arah:

- Forward: dari awal ke akhir sekuens
- Backward: dari akhir ke awal sekuens

Ini memungkinkan model untuk mengakses informasi dari masa depan dan masa lalu.

In [ ]:
# Bidirectional LSTM
bidirectional_model = keras.models.Sequential([
    keras.layers.Bidirectional(
        keras.layers.LSTM(20, return_sequences=True),
        input_shape=[None, 1]
    ),
    keras.layers.Bidirectional(keras.layers.LSTM(20)),
    keras.layers.Dense(1)
])

bidirectional_model.compile(optimizer='adam', loss='mse')
bidirectional_model.summary()

##5. Sequence-to-Sequence Models
###Encoder-Decoder Architecture
Model encoder-decoder digunakan untuk tugas di mana input dan output memiliki panjang yang berbeda:


In [ ]:
# Encoder-Decoder untuk sequence prediction
def build_seq2seq_model(n_steps_in=50, n_steps_out=10, n_features=1):
    # Encoder
    encoder_inputs = keras.layers.Input(shape=(n_steps_in, n_features))
    encoder_lstm = keras.layers.LSTM(50, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = keras.layers.Input(shape=(n_steps_out, n_features))
    decoder_lstm = keras.layers.LSTM(50, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(n_features)
    decoder_outputs = decoder_dense(decoder_outputs)

    model = keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model

seq2seq_model = build_seq2seq_model()
seq2seq_model.compile(optimizer='adam', loss='mse')

##6. Attention Mechanism
###Konsep Attention
Attention mechanism memungkinkan model untuk "memperhatikan" bagian-bagian penting dari input sequence:


In [ ]:
# Simple Attention Layer
class AttentionLayer(keras.layers.Layer):
    def __init__(self, n_units):
        super(AttentionLayer, self).__init__()
        self.n_units = n_units
        self.W = keras.layers.Dense(n_units)
        self.V = keras.layers.Dense(1)

    def call(self, inputs):
        # inputs shape: (batch_size, time_steps, features)
        score = self.V(tf.nn.tanh(self.W(inputs)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * inputs
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

# Model dengan Attention
def build_attention_model():
    inputs = keras.layers.Input(shape=[None, 1])
    lstm_out = keras.layers.LSTM(50, return_sequences=True)(inputs)
    context_vector, attention_weights = AttentionLayer(50)(lstm_out)
    outputs = keras.layers.Dense(1)(context_vector)

    model = keras.models.Model(inputs, outputs)
    return model

attention_model = build_attention_model()
attention_model.compile(optimizer='adam', loss='mse')

##7. Evaluasi dan Perbandingan Model

In [ ]:
# Fungsi untuk evaluasi model
def evaluate_models(*models, model_names):
    results = {}
    for model, name in zip(models, model_names):
        loss = model.evaluate(X_test, y_test, verbose=0)
        predictions = model.predict(X_test[:100])
        results[name] = {
            'loss': loss,
            'predictions': predictions
        }
    return results

# Visualisasi hasil prediksi
def plot_predictions(X_test, y_test, predictions, model_name):
    plt.figure(figsize=(12, 6))
    for i in range(3):  # Plot 3 contoh
        plt.subplot(1, 3, i+1)
        plt.plot(X_test[i].flatten(), label='Input')
        plt.axhline(y=y_test[i], color='red', linestyle='--', label='True')
        plt.axhline(y=predictions[i], color='blue', linestyle='--', label='Predicted')
        plt.title(f'{model_name} - Sample {i+1}')
        plt.legend()
    plt.tight_layout()
    plt.show()

##Kesimpulan

Chapter 15 membahas berbagai teknik untuk memproses data sekuensial:

- RNN Dasar: Cocok untuk sekuens pendek tetapi mengalami masalah vanishing gradient
- LSTM: Mengatasi masalah vanishing gradient dengan gating mechanism yang kompleks
- GRU: Versi sederhana LSTM dengan performa yang sebanding
- CNN 1D: Alternatif yang lebih cepat untuk mendeteksi pola lokal dalam sekuens
- Bidirectional RNN: Menggunakan informasi dari kedua arah untuk prediksi yang lebih baik
- Seq2Seq: Untuk tugas dengan input dan output yang berbeda panjang
- Attention: Memungkinkan model fokus pada bagian penting dari input
<br><br>

###Pemilihan arsitektur tergantung pada:

- Panjang sekuens
- Kompleksitas pola
- Ketersediaan data
- Kebutuhan kecepatan training dan inference
<br><br>

###Tips Praktis:

- Gunakan LSTM/GRU untuk dependensi jangka panjang
- CNN 1D untuk pola lokal dan kecepatan
- Bidirectional untuk akses informasi masa depan
- Attention untuk sekuens yang sangat panjang
- Kombinasi CNN + RNN untuk hasil terbaik